In [15]:
from typing import Tuple
import tifffile
import os
from os.path import join
import numpy as np
import SimpleITK as sitk
from natsort import natsorted

def convert_tiff_to_nifti(load_dir: str, save_dir: str, spacing: Tuple[float, float, float]) -> None:
    """
    Converts a set of TIFF image slices to a NIFTI image.

    Args:
        load_dir (str): Path to the TIFF files.
        save_dir (str): Path to where the NIFTI image should be saved.
        spacing (Tuple[float, float, float]): The image spacing given as three values.
    """
    name = os.path.basename(os.path.normpath(load_dir))
    filepaths = load_filepaths(load_dir, extension=['tif', 'tiff', 'TIF', 'TIFF'])
    image = tifffile.imread(filepaths)
    os.makedirs(save_dir, exist_ok=True)
    save_nifti(join(save_dir, name + '.nii.gz'), image, spacing=spacing)

def load_filepaths(load_dir: str, extension: str = None, return_path: bool = True, return_extension: bool = True) -> np.ndarray:
    """
    Given a directory path, returns an array of file paths with the specified extension.

    Args:
        load_dir: The directory containing the files.
        extension: A string or list of strings specifying the file extension(s) to search for. Optional.
        return_path: If True, file paths will include the directory path. Optional.
        return_extension: If True, file paths will include the file extension. Optional.

    Returns:
        An array of file paths.
    """
    filepaths = []
    if isinstance(extension, str):
        extension = tuple([extension])
    elif isinstance(extension, list):
        extension = tuple(extension)
    elif extension is not None and not isinstance(extension, tuple):
        raise RuntimeError("Unknown type for argument extension.")

    if extension is not None:
        extension = list(extension)
        for i in range(len(extension)):
            if extension[i][0] != ".":
                extension[i] = "." + extension[i]
        extension = tuple(extension)

    for filename in os.listdir(load_dir):
        if extension is None or str(filename).endswith(extension):
            if not return_extension:
                if extension is None:
                    filename = filename.split(".")[0]
                else:
                    for ext in extension:
                        if str(filename).endswith((ext)):
                            filename = str(filename)[:-len(ext)]
            if return_path:
                filename = join(load_dir, filename)
            filepaths.append(filename)
    filepaths = np.asarray(filepaths)
    filepaths = natsorted(filepaths)

    return filepaths

def save_nifti(filename: str,
               image: np.ndarray,
               spacing: Tuple[float] = None,
               affine: np.ndarray = None,
               header: dict = None,
               is_seg: bool = False,
               dtype: np.dtype = None) -> None:
    """
    Saves a NIfTI file to disk.

    Args:
        filename (str): The filename of the NIfTI file to save.
        image (np.ndarray): The image data to save.
        spacing (Tuple[float], optional): The voxel spacing in mm. Defaults to None.
        affine (np.ndarray, optional): The affine transform matrix. Defaults to None.
        header (dict, optional): A dictionary of meta-data to save. Defaults to None.
        is_seg (bool, optional): Whether the image is a segmentation. If True, the image is rounded to the nearest
            integer and converted to int16 data type. Defaults to False.
        dtype (np.dtype, optional): The data type to save the image as. If None, the data type is determined by the
            image data. Defaults to None.

    Returns:
        None
    """
    if is_seg:
        image = np.rint(image)
        if dtype is None:
            image = image.astype(np.int16)  # In special cases segmentations can contain negative labels, so no np.uint8 by default

    if dtype is not None:
        image = image.astype(dtype)

    image = sitk.GetImageFromArray(image)

    if header is not None:
        [image.SetMetaData(key, header[key]) for key in header.keys()]

    if spacing is not None:
        image.SetSpacing(spacing)

    if affine is not None:
        pass  # How do I set the affine transform with SimpleITK? With NiBabel it is just nib.Nifti1Image(img, affine=affine, header=header)

    sitk.WriteImage(image, filename)
    
def setup_paths(dir_location, is_original_data):
    if dir_location.lower() == 'internal':
        base_path = r'C:\Senior_Design'
    elif dir_location.lower() == 'external':
        base_path = r'D:\Senior_Design'
    elif dir_location.lower() == 'cloud':
        base_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files'
    elif dir_location.lower() == 'refine':
        base_path = r'D:\Darren\Files'
    else:
        raise ValueError('Invalid directory location type')
    
    base_tiff_path = os.path.join(base_path, 'database')
    if is_original_data:
        tiff_path = os.path.join(base_tiff_path, 'orignal_dataset', 'instance', 'tiff')
    else:
        tiff_path = os.path.join(base_tiff_path, 'tablet_dataset', 'instance', 'tiff')
    
    nifti_path = tiff_path.replace('tiff', 'nifti')

    print('Paths set')
    return tiff_path, nifti_path

In [ ]:
import os

tiff_path, nifti_path = setup_paths('refine', False)
img_names = ['2_Tablet_Aug1', '2_Tablet_Aug2', '2_Tablet_Aug3', '2_Tablet_Aug4', '2_Tablet_Aug5',
               '4_GenericD12_Aug1', '4_GenericD12_Aug2', '4_GenericD12_Aug3', '4_GenericD12_Aug4', '4_GenericD12_Aug5',
               '5_ClaritinD12_Aug1', '5_ClaritinD12_Aug2', '5_ClaritinD12_Aug3', '5_ClaritinD12_Aug4', '5_ClaritinD12_Aug5']
voxel_spacings = [0.0129988, 0.0129988, 0.0129988, 0.0129988, 0.0129988,
                  0.0129980, 0.0129980, 0.0129980, 0.0129980, 0.0129980,
                  0.0121229, 0.0121229, 0.0121229, 0.0121229, 0.0121229]

for img_name, voxel_spacing in zip(img_names, voxel_spacings):
    input_path = os.path.join(tiff_path, img_name)
    output_path = os.path.join(nifti_path, img_name)

    convert_tiff_to_nifti(input_path, output_path, (voxel_spacing, voxel_spacing, voxel_spacing))
    print('Conversion for ' + img_name + ' is complete.')

Paths set
Conversion for 2_Tablet_Aug1 is complete.
Conversion for 2_Tablet_Aug2 is complete.
Conversion for 2_Tablet_Aug3 is complete.
Conversion for 2_Tablet_Aug4 is complete.
Conversion for 2_Tablet_Aug5 is complete.
Conversion for 4_GenericD12_Aug1 is complete.
Conversion for 4_GenericD12_Aug2 is complete.
Conversion for 4_GenericD12_Aug3 is complete.
Conversion for 4_GenericD12_Aug4 is complete.
Conversion for 4_GenericD12_Aug5 is complete.
Conversion for 5_ClaritinD12_Aug1 is complete.
Conversion for 5_ClaritinD12_Aug2 is complete.
Conversion for 5_ClaritinD12_Aug3 is complete.
Conversion for 5_ClaritinD12_Aug4 is complete.
Conversion for 5_ClaritinD12_Aug5 is complete.
